In [1]:
import json
import csv
import pandas as pd
import numpy as np
import glob
import os
import sys
from tools.data2vec_norm_2i import Vecter
import random

In [5]:
with open("./kif_data.json", mode="r", encoding="utf-8") as f:
    origin = json.load(f)
print(origin[0]["SFEN"][20])

ln1gkg1nl/1r7/p1p1p1spp/1p1pspp2/7P1/2P1P1P2/PP1P1PS1P/1SKG3R1/LN3G1NL w Bb 22


In [7]:
print(origin[0]["SFEN"][20], origin[0]["対局日"], origin[0]["先手"], origin[0]["後手"],
                   origin[0]["評価値"][20], origin[0]["時間推移"][20])
Converter = Vecter(origin[0]["SFEN"][20], origin[0]["対局日"], origin[0]["先手"], origin[0]["後手"],
                   origin[0]["評価値"][20], origin[0]["時間推移"][20])

ln1gkg1nl/1r7/p1p1p1spp/1p1pspp2/7P1/2P1P1P2/PP1P1PS1P/1SKG3R1/LN3G1NL w Bb 22 ['2021', '06', '22'] 307 194 0.0104 [0.982037037037037, 0.9879629629629629]
[[-0.7        -0.6         0.         -0.4        -0.1        -0.4
   0.         -0.6        -0.7         0.          0.          0.
   0.          0.        ]
 [ 0.         -0.2         0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [-0.8         0.         -0.8         0.         -0.8         0.
  -0.5        -0.8        -0.8         0.          0.          0.
   0.          0.        ]
 [ 0.         -0.8         0.         -0.8        -0.5        -0.8
  -0.8         0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.8         0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.8         0.       

In [12]:
pd.DataFrame(Converter.vec[0])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.700000,-0.6,0.0,-0.4,-0.1,-0.4,0.0,-0.6,-0.7,0.0,0.0,0.0,0.0,0.0
1,0.000000,-0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.800000,0.0,-0.8,0.0,-0.8,0.0,-0.5,-0.8,-0.8,0.0,0.0,0.0,0.0,0.0
3,0.000000,-0.8,0.0,-0.8,-0.5,-0.8,-0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.8,0.0,0.8,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.800000,0.8,0.0,0.8,0.0,0.8,0.5,0.0,0.8,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.5,0.1,0.4,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0
8,0.700000,0.6,0.0,0.0,0.0,0.4,0.0,0.6,0.7,0.0,0.0,0.0,0.0,0.0
9,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
pd.DataFrame(Converter.vec[1])

,0
0,0.210526
1,0.279412
2,0.010400
3,0.982037
4,0.987963


In [13]:
f = open("annotation/35anno_devidecomplate.csv", mode="w+", encoding="utf-8", newline="")
ftrain = open("annotation/35anno_train_devidecomplate.csv", mode="w+", encoding="utf-8", newline="")
ftest = open("annotation/35anno_test_devidecomplate.csv", mode="w+", encoding="utf-8", newline="")
writer = csv.writer(f, delimiter=",")
wtrain = csv.writer(ftrain, delimiter=",")
wtest = csv.writer(ftest, delimiter=",")
memo = {}
for data in origin:
    date = data["対局日"]
    black, white = data["先手"], data["後手"]
    winner = 0 if data["勝者"]=="先手" else 1
    path_dir = "datasets_2i/"+"-".join(date)+"-"+str(black)+"-"+str(white)
    memo[path_dir] = 0
    choises = random.choices(range(35, len(data["SFEN"])-5), k=5)
    #choise, test_num = choises[:4], choises[4]
    cnt = 0
    print(len(data["SFEN"]))
    print(choises)
    """#既存データセットに対してアノテーションファイルを生成する際はコメントアウトする
    try:
        os.mkdir(path_dir)
    except FileExistsError:
        #memo[path_dir] = memo[path_dir] + 1
        #path_dir = path_dir+"-"+str(memo[path_dir])
        #os.mkdir(path_dir)
        continue
    """
    for idx in range(len(data["SFEN"])):
        sfen = data["SFEN"][idx]
        score = data["評価値"][idx]
        time = data["時間推移"][idx]
        path = path_dir+"/"+str(idx+1)+".csv"
        vec = Vecter(sfen, date, black, white, score, time).vec
        df = pd.DataFrame(vec)
        #df.to_csv(path, header=False, index=False)
        writer.writerow([path, winner])
        #if idx in choise:
        if (idx in choises) and (cnt%20 != 0):
            #print(path)
            wtrain.writerow([path, winner])
        #elif idx == test_num:
        elif (idx in choises) and (cnt%20 == 0):
            print(path)
            wtest.writerow([path, winner])
        cnt = cnt+1
f.close()
ftrain.close()
ftest.close()

87
[42, 63, 71, 49, 62]
114
[42, 68, 66, 39, 85]
95
[84, 75, 36, 55, 44]
161
[42, 65, 114, 141, 66]
91
[82, 81, 81, 70, 36]
111
[38, 105, 68, 103, 73]
142
[110, 52, 87, 84, 41]
93
[41, 39, 71, 38, 79]
74
[61, 56, 59, 52, 43]
109
[103, 72, 70, 42, 67]
103
[67, 80, 56, 41, 43]
datasets_norm/2015-02-11-235-175/81.csv
96
[63, 78, 81, 47, 89]
107
[55, 85, 75, 40, 42]
datasets_norm/2015-03-08-235-175/41.csv
179
[45, 168, 66, 61, 145]
124
[96, 85, 45, 114, 74]
88
[36, 42, 65, 80, 53]
datasets_norm/2016-02-11-263-235/81.csv
66
[48, 58, 40, 58, 53]
datasets_norm/2016-02-20-235-263/41.csv
170
[60, 121, 41, 109, 49]
datasets_norm/2016-03-06-263-235/61.csv
125
[69, 90, 76, 117, 52]
112
[78, 86, 96, 64, 99]
157
[110, 148, 113, 116, 115]
131
[82, 105, 40, 93, 62]
datasets_norm/2017-02-18-235-291/41.csv
121
[45, 74, 111, 64, 43]
139
[81, 84, 55, 68, 48]
104
[85, 96, 79, 81, 40]
datasets_norm/2017-12-27-298-276/41.csv
189
[154, 62, 152, 86, 75]
123
[42, 71, 57, 56, 110]
103
[91, 81, 62, 54, 58]
161
[1

In [16]:
f.close()
ftrain.close()
ftest.close()